In [2]:
from datasets import load_dataset, Dataset, DatasetDict

In [3]:
dataset = load_dataset("whitefox44/ReflectionGPT4")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
from pprint import pprint
pprint(dataset["train"][0])

{'input': '',
 'instruction': 'Give three tips for staying healthy.',
 'output': '\n'
           '1. Eat a balanced diet and make sure to include plenty of fruits '
           'and vegetables. \n'
           '2. Exercise regularly to keep your body active and strong. \n'
           '3. Get enough sleep and maintain a consistent sleep schedule.',
 'reflection': "The AI's response to the given instruction is generally "
               'accurate and relevant, providing three clear tips for staying '
               'healthy. However, there is still room for improvement in terms '
               'of depth, specificity, and personalization. To enhance the '
               'quality of the response, the AI could consider the following '
               'improvements: \n'
               '\n'
               '1. Provide more depth and detail: The tips given are quite '
               'basic and common. To make the response more valuable, the AI '
               'could expand on each tip with more 

In [5]:
# Filter examples with inputs
dataset = dataset.filter(lambda x: not x["input"])

len(dataset["train"])

Filter:   0%|          | 0/143 [00:00<?, ? examples/s]

99

In [6]:
# Remove <|endoftext|> from the end of the examples
dataset = dataset.map(lambda x: {
    "output": x["output"].replace("<|endoftext|>", "").strip(),
    "reflection": x["reflection"].replace("<|endoftext|>", "").strip(),
})

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [7]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)


In [8]:
def to_chatml(row):
    instruction = row["instruction"]
    output = row["output"]
    reflection = row["reflection"]
    newline = "\n"
   
    # User message
    user_message = (
        "Analyze the following conversation between a User and an LLM-based AI Assistant."
        " Then reflect on the AI's response in the conversation."
        f"{newline}{newline}"
        f"User: {instruction}"
        f"{newline}{newline}"
        f"AI Assistant: {output}"
    )

    # Turn into chatml
    chatml = [
        person(user_message),
        thought(f"Reflection:{newline}{newline}{reflection}"),
    ]

    return dict(chatml=chatml)


In [9]:
dataset = dataset.map(to_chatml)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [10]:
dataset["train"][0]

{'output': '1. Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 'input': '',
 'instruction': 'Give three tips for staying healthy.',
 'reflection': "The AI's response to the given instruction is generally accurate and relevant, providing three clear tips for staying healthy. However, there is still room for improvement in terms of depth, specificity, and personalization. To enhance the quality of the response, the AI could consider the following improvements: \n\n1. Provide more depth and detail: The tips given are quite basic and common. To make the response more valuable, the AI could expand on each tip with more specific advice, such as recommended daily servings of fruits and vegetables, types of exercises for different fitness levels, or the optimal sleep duration for various age groups. \n2. Offer evidence or expert advice: C

In [11]:
dataset = dataset.remove_columns(["reflection", "instruction", "input", "output"])

In [13]:
dataset.push_to_hub("diwank/reflection_gpt4-chatml", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]